<h1>Principal Component Analysis</h1>
<b>It is a way of reducing the number of dimensions without losing much variance.</b>
<i></i>

In [48]:
import numpy as np
import pandas as pd
import matplotlib as pp
%matplotlib inline

In [49]:
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer

In [50]:
cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [51]:
df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


<b>For PCA to work properly we should do mean normalisation and feature scaling before fitting the data</b>

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
scaler = StandardScaler()
scaler.fit(df)
df_scaled = scaler.transform(df)


In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_scaled, cancer['target'], test_size=0.3, random_state=101)

In [55]:
# run pca without n_components to get all the eigenvectors and then we can check the variance ratios
# to determine number of components to consider to achieve a certain desired variance.
pca = PCA() 
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [56]:
# Now transform our n-dimensional training data into k dimensional data.
X_train_pca = pca.transform(X_train)

<b>Determine number of PCA components to retain, if we want to preserve 90% variance</b>

In [57]:
index = np.argmax(np.cumsum(pca.explained_variance_ratio_) > .90) 
np.cumsum(pca.explained_variance_ratio_[:index + 1])

array([ 0.4451123 ,  0.64923107,  0.73065293,  0.79744894,  0.85361861,
        0.89423853,  0.9145795 ])

In [58]:
# This indicate the value of k-components to use for the expected variance 
# in this case we should take 7 components
index 

6

<b>Now create the PCA again with required number of components and fit our training data to it</b>

In [59]:
pca = PCA(n_components=index + 1)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_train_pca.shape

(398, 7)

In [60]:
# components kept
evecs = pca.components_

In [61]:
evecs.shape

(7, 30)

In [62]:
print('Original Data shape ',X_train.shape)
print('Transformed Data shape', X_train_pca.shape)

Original Data shape  (398, 30)
Transformed Data shape (398, 7)


<b>Get back approximate values from our reduced dimension to original dimensions</b>

In [63]:
df_approx = np.dot(X_train_pca, pca.components_)
df_approx.shape

(398, 30)

In [64]:
# need to reverse the scaling to get back original values
df_approx = scaler.inverse_transform(df_approx)

<b>Compare original and values reconstructed from PCA components

In [65]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [66]:
df_approx = pd.DataFrame(df_approx, columns=cancer['feature_names'])

In [67]:
df_approx.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,12.854797,22.011303,80.622153,517.895591,0.063109,0.011020,-0.030442,-0.006763,0.141342,0.050775,...,14.230656,28.416221,90.197073,648.046480,0.083932,0.055559,0.005329,0.026088,0.229343,0.058001
1,14.027096,15.080167,92.700349,688.004855,0.105967,0.144973,0.142552,0.067158,0.202869,0.070730,...,16.194965,18.635434,108.668475,941.874649,0.137887,0.302601,0.332974,0.127188,0.302513,0.094379
2,15.014173,21.489374,99.027728,722.874257,0.103840,0.145923,0.128836,0.068066,0.199519,0.066142,...,18.092024,30.168554,121.322003,1048.578489,0.152002,0.432624,0.455993,0.166877,0.366441,0.103511
3,15.329401,20.000053,99.175788,757.284911,0.090718,0.080491,0.075345,0.047270,0.156104,0.057059,...,17.567855,26.790681,114.808408,1011.458688,0.125602,0.194802,0.243939,0.114907,0.233331,0.073076
4,10.605573,19.177216,67.118755,322.285982,0.085944,0.055664,0.010170,0.005984,0.165560,0.062669,...,11.405942,24.898924,72.696017,350.740878,0.116204,0.116976,0.066259,0.036847,0.245547,0.076202


<b>Now build a Logistic Regression model for our data with reduced dimensions</b>

In [68]:
# use the transform values to build a classification model
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()

In [69]:
reg.fit(X_train_pca, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [70]:
X_test_pca = pca.transform(X_test)
y_pred = reg.predict(X_test_pca)

In [71]:
X_test_pca.shape

(171, 7)

In [72]:
from sklearn.metrics import confusion_matrix, classification_report

In [73]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))

[[ 63   3]
 [  3 102]]
             precision    recall  f1-score   support

          0       0.95      0.95      0.95        66
          1       0.97      0.97      0.97       105

avg / total       0.96      0.96      0.96       171

